# **💡0️⃣ [ 아래 코드를 실행하여 조별 번호를 입력하시오.]💡**

*   1조: 박유진, 김서연, 최진
*   2조: 서민경, 이유진, 최범영
*   3조: 정은주, 김장환, 권진경
*   4조: 정진교, 장채은, 조성환
*   5조: 전승재, 신은호, 김윤희
*   6조: 정현서, 진경은, 이정민
*   7조: 이성지, 김내경



In [1]:
print("[예]조의 번호를 입력하시오: 1")
team_idx = int(input(">> 조의 번호를 입력하시오: "))
print(f">> {team_idx}조 입니다.")
team_idx = 'team'+str(team_idx)

[예]조의 번호를 입력하시오: 1
>> 6조 입니다.


In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from tqdm import tqdm

import torchvision
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torchvision.datasets.utils import download_url, extract_archive
import random
import matplotlib.pyplot as plt
import numpy as np
import os

%matplotlib inline
from torchinfo import summary

URL = "https://github.com/JanghunHyeon/AISW4202-Project/releases/download/v.1.1.0/project_dataset.zip"
ROOT = "./content/data"
ZIP_PATH = os.path.join(ROOT, "project_dataset.zip")
OUT_DIR  = os.path.join(ROOT, "project_dataset")

os.makedirs(ROOT, exist_ok=True)
download_url(URL, root=ROOT, filename="project_dataset.zip")
extract_archive(ZIP_PATH, OUT_DIR)

device = 'cuda' if torch.cuda.is_available() else 'cpu'


# Reproduce를 위한 Seed 고정
seed_id = 777
deterministic = True

random.seed(seed_id)
np.random.seed(seed_id)
torch.manual_seed(seed_id)
if device =='cuda':
    torch.cuda.manual_seed_all(seed_id)
if deterministic:
	torch.backends.cudnn.deterministic = True
	torch.backends.cudnn.benchmark = False

# **💡1️⃣ [ Design your custom model ]💡**
  ## 아래 코드를 수정하여 본인의 모델을 만들어 보세요.

In [ ]:
class MyCustomModel(nn.Module):
    def __init__(self, num_classes=10, init_weights=True):
        super().__init__()

        # 48x48
        self.conv1 = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=7, stride=1, padding=3),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        )

        # 24x24
        self.down_sample_layer1 = nn.Sequential(
            nn.Conv2d(64, 128, kernel_size=1, stride=2),
            nn.BatchNorm2d(128),
            nn.ReLU()
        )

        self.conv2_1 = nn.Sequential(nn.Conv2d(64, 64, kernel_size=1),
                                     nn.BatchNorm2d(64),
                                     nn.ReLU(),

                                     nn.Conv2d(64, 64, kernel_size=3, padding=1, stride=2),
                                     nn.BatchNorm2d(64),
                                     nn.ReLU(),

                                     nn.Conv2d(64, 128, kernel_size=1),
                                     nn.BatchNorm2d(128),
                                     )
        self.relu = nn.ReLU()


        # 12x12
        self.down_sample_layer2 = nn.Sequential(nn.Conv2d(128, 256, kernel_size=1, stride=2),
                                                nn.BatchNorm2d(256),
                                                nn.ReLU())

        self.conv3_1 = nn.Sequential(nn.Conv2d(128, 64, kernel_size=1),
                                     nn.BatchNorm2d(64),
                                     nn.ReLU(),

                                     nn.Conv2d(64, 64, kernel_size=3, padding=1, stride=2),
                                     nn.BatchNorm2d(64),
                                     nn.ReLU(),

                                     nn.Conv2d(64, 256, kernel_size=1),
                                     nn.BatchNorm2d(256),
                                     )
        # 6x6
        self.conv3_2 = nn.Sequential(nn.Conv2d(256, 128, kernel_size=1),
                                     nn.BatchNorm2d(128),
                                     nn.ReLU(),

                                     nn.Conv2d(128, 128, kernel_size=3, padding=1),
                                     nn.BatchNorm2d(128),
                                     nn.ReLU(),

                                     nn.Conv2d(128, 256, kernel_size=1),
                                     nn.BatchNorm2d(256),
                                     )

        # 6x6
        self.global_pool = nn.AvgPool2d(kernel_size=6)
        self.fc = nn.Linear(256, num_classes)

        if init_weights:
            self._initialize_weights()

    def forward(self, x):

        x = self.conv1(x)

        identity1 = self.down_sample_layer1(x)
        out = self.conv2_1(x)
        out = identity1 + out
        out = self.relu(out)

        identity3 = self.down_sample_layer2(out)
        out = self.conv3_1(out)
        out = identity3 + out
        out = self.relu(out)

        identity4 = out
        out = self.conv3_2(out)
        out = out+identity4
        out = self.relu(out)

        pool = self.global_pool(out)
        fc = pool.view(pool.size(0), -1)
        fc = self.fc(fc)

        return fc

    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)

In [4]:
model = MyCustomModel(num_classes=15).to(device)

inputs = torch.Tensor(1,3,48,48).to(device)
out = model(inputs)
print(out)

tensor([[-1.0525,  0.4546,  0.4724,  0.7954,  0.0539,  0.0900, -0.0849, -0.5943,
          0.0419,  0.0494,  0.2299, -0.3144,  0.2880, -0.1051, -0.9304]],
       device='cuda:0', grad_fn=<AddmmBackward0>)


In [5]:
summary(model, (1, 3, 48, 48))

Layer (type:depth-idx)                   Output Shape              Param #
MyCustomModel                            [1, 15]                   --
├─Sequential: 1-1                        [1, 64, 24, 24]           --
│    └─Conv2d: 2-1                       [1, 64, 48, 48]           9,472
│    └─BatchNorm2d: 2-2                  [1, 64, 48, 48]           128
│    └─ReLU: 2-3                         [1, 64, 48, 48]           --
│    └─MaxPool2d: 2-4                    [1, 64, 24, 24]           --
├─Sequential: 1-2                        [1, 128, 12, 12]          --
│    └─Conv2d: 2-5                       [1, 128, 12, 12]          8,320
│    └─BatchNorm2d: 2-6                  [1, 128, 12, 12]          256
│    └─ReLU: 2-7                         [1, 128, 12, 12]          --
├─Sequential: 1-3                        [1, 128, 12, 12]          --
│    └─Conv2d: 2-8                       [1, 64, 24, 24]           4,160
│    └─BatchNorm2d: 2-9                  [1, 64, 24, 24]           128
│  

# **💡2️⃣[ Dataset Processing ]💡**

## Dataset for Final Project
- Large-scale RGB image dataset
  - Training set: about 75,000 images
  - Validation set: 900 images
  
- Dataset specification
  - image size: 48 x 48
  - RGB scale image (3 channel)
  - 15 classes

In [6]:
transform_train = transforms.Compose([
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomRotation(degrees=10),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std =[0.229, 0.224, 0.225]
    ),
])

transform_val = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std =[0.229, 0.224, 0.225]
    ),
])

data_dir = os.path.join(OUT_DIR, "data/ProjectDataset")
trainset = ImageFolder(os.path.join(data_dir, "train"), transform=transform_train)
valset = ImageFolder(os.path.join(data_dir, "val"), transform=transform_val)

trainloader = DataLoader(trainset, batch_size=256, shuffle=True, num_workers=6)
valloader = DataLoader(valset, batch_size=64, shuffle=False, num_workers=6)

# **💡3️⃣[ Train your custom model ]💡**


In [7]:
criterion = nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr = 0.005)

lr_sche = optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.9)

In [8]:
print(len(trainloader))
epochs = 50

for epoch in range(epochs):  # loop over the dataset multiple times
    model.train()
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs
        inputs, labels = data
        inputs = inputs.to(device)
        labels = labels.to(device)

        # zero the parameter gradients
        optimizer.zero_grad()
        # forward + backward + optimize
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 50 == 49:    # print every 50 iterations
            print('[%d, %5d] loss: %.3f' %(epoch + 1, i + 1, running_loss / 50))
            running_loss = 0.0

    lr_sche.step()

    correct = 0
    total = 0
    model.eval()
    with torch.no_grad():
        for data in valloader:
            images, labels = data
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images)

            _, predicted = torch.max(outputs.data, 1)

            total += labels.size(0)

            correct += (predicted == labels).sum().item()

    print('Accuracy of the network on the 1,200 validation images: %.3f %%' % (100 * correct / total))


print('Finished Training')

288
[1,    50] loss: 2.014
[1,   100] loss: 1.704
[1,   150] loss: 1.558
[1,   200] loss: 1.469
[1,   250] loss: 1.408
Accuracy of the network on the 1,200 validation images: 54.778 %
[2,    50] loss: 1.266
[2,   100] loss: 1.229


KeyboardInterrupt: 

# 💡4️⃣ [Model Evaluation]💡
* Validation Set을 이용한 성능 검증

In [13]:
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for data in valloader:
        images, labels = data
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)

        _, predicted = torch.max(outputs.data, 1)

        total += labels.size(0)

        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 1,200 validation images: %.3f %%' % (100 * correct / total))

Accuracy of the network on the 1,200 validation images: 76.000 %


# 💡5️⃣ [Save Model]💡
1. 아래 코드를 동작하면 Google 인증 팝업이 뜨고 인증을 요청.
2. 인증후, Drive에 Final_Project/Model이라는 폴더가 생성됨.
3. 생성된 폴더 안에 **model_{team_idx}.pt** 라는 파일이 생성됨.
4. 생성된 파일과 코드를 다운받아 LMS에 제출




In [16]:
fname = f"model_{team_idx}.pt"

model.eval()
example_input = torch.randn(1,3,48,48).to(device)
traced_script = torch.jit.trace(model, example_input)
traced_script.save(fname)


# 구축한 데이터셋 Google Drive로 저장
# #from google.colab import drive
# import shutil

# # 1) Drive 마운트
# #drive.mount('/content/drive')
# data_dir = './content'
# out_dirs = './content/Final_Project/model'
# os.makedirs(out_dirs, exist_ok=True)

# shutil.copy(os.path.join(data_dir, fname), os.path.join(out_dirs, fname))
# print("Copied to Drive → MyDrive")